In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
data = pd.read_csv('hadoop.csv')

In [3]:
data = data.drop(['idx','access_date','access_time','access_page','access_referer'],axis=1)

In [4]:
unique_ip = data[data['item_type']=='cart']['access_ip'].unique()

# Make Dict_ip insert key is ip item is values

In [5]:
dict_ip = dict.fromkeys(unique_ip)   #dict key 생성

for i in dict_ip.keys():
    dict_ip[i] = []

#-----------------------------------------------#

for i in range(len(data)):          #dict value 삽입
    if (data['access_ip'].iloc[i] in dict_ip.keys()):
        key = data['access_ip'].iloc[i]
        dict_ip[key].append(data['item_id'].iloc[i])

In [6]:
for i,j in dict_ip.items():
    dict_ip[i] = list(map(str,j))        #dict value를 string으로 매핑
    
#------------------------------------------------#
#dict의 벨류가 4 미만이면 삭제
dict_ip = {key:value for key, value in dict_ip.items() if len(value) >4} 

In [7]:
cart_item = []       #dict의 마지막 아이템을 카트 아이템배열에 넣음
for i,j in dict_ip.items():
    cart_item.append(j[-1])

In [8]:
columns_name = []      #column이름을 string으로 변환
for i in range(14):
    columns_name.append(str(i))

# Make DataFrame New

In [9]:
df = pd.DataFrame.from_dict(dict_ip,orient='index',columns=columns_name)

In [10]:
mask = df.applymap(lambda x: x is None)         #None값들 삭제
cols = df.columns[(mask).any()]
for col in df[cols]:
    df.loc[mask[col], col] = ''

In [11]:
item_list = []                 #item들을 뽑아서 한 row씩 한 리스트로 묶어 배열 생성
for i in range(len(df)):
    item_list.append(df.iloc[i,:].tolist())

# Word2Vec

In [12]:
df1 = df.head(3032)
df = df.append(df1)

In [13]:
from gensim.models import Word2Vec
from gensim.test.utils import common_texts, get_tmpfile

In [14]:
path = get_tmpfile('word2vec.model')

In [15]:
model = Word2Vec(item_list, size=1 ,window=5, min_count=1,workers=5)
model.save('word2vec.model')

In [16]:
model = Word2Vec.load('word2vec.model')

In [17]:
word_vectors = model.wv  
vocabs = word_vectors.vocab.keys() #vocabs has keys that is item number

#word_vectors_list has values that is vector value from item number
word_vectors_list = [word_vectors[v] for v in vocabs] 

In [18]:
for i in range(len(df)):            #word2vec value를 df로 넣음
    for j in range(len(df.iloc[i])):
        key=str(df.iloc[i,j])
        df.iloc[i,j] = word_vectors[key].tolist()[0]

In [19]:
df = df.apply(pd.to_numeric)

# Data split && Learning

In [20]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
from heapq import nsmallest
import time

In [21]:
X_data = df.drop('13',axis=1)
y_data = df['13']

# Result

In [22]:
seed = 6
test_size = 0.2
X_train,X_test,y_train,y_test = train_test_split(X_data,y_data,test_size=test_size, random_state=seed)
model = xgb.XGBClassifier(n_estimators=1000,max_depth=8,learning_rate=0.1,subsample=0.5)
time1 = time.time()
model.fit(X_train,y_train)
time2 = time.time()
print(time2-time1)

750.0443375110626


In [25]:
avg_time = []
avg_acu = []

for i in range(10):
    print(i+1, "번째 추천 시작")
    
    result = pd.DataFrame()
    start = time.time()
    seed = i
    X_train,X_test,y_train,y_test = train_test_split(X_data,y_data,test_size=test_size, random_state=seed)
    prediction = model.predict(X_test)
    end = time.time()
    print(i+1, "번째 추천(2000건)에 걸린 시간(초):", end-start)
    print(i+1, "번째 데이터 한개당 걸린 시간 :", (end-start)/2000)
    
    result['predicted'] = prediction
    result['Y_test'] = y_test.tolist()
    
    #예측값과 근사한 값 5개 출력
    vector_list = []

    for i in range(len(result)):
        vector_list.append(nsmallest(6, word_vectors_list, key=lambda x: abs(x-result['predicted'].iloc[i])))
    
    result['Nearby5'] = pd.Series(vector_list)
    for i in range(len(result)):
        result['Nearby5'].iloc[i] = vector_list[i]
    
    T_F = []
    for i in range(len(result)):
        if result['Y_test'].iloc[i] in result['Nearby5'].iloc[i]:
            T_F.append('True')
        else:
            T_F.append('False')
    
    result['T_F'] = T_F
    
    print("예측한 값과 가까운 아이템 5개의 Accuracy :", len(result[result['T_F']=='True'])/len(result))
    
    
    avg_acu.append(len(result[result['T_F']=='True'])/len(result))

    
    avg_time.append((end - start)/2000)
    print("----------------------------------------------------")
    
print("추천 정확도의 평균 :", sum(avg_acu)/len(avg_acu))
print("데이터 한개당 걸린 시간의 평균 :", sum(avg_time)/len(avg_time))

1 번째 추천 시작
1 번째 추천(2000건)에 걸린 시간(초): 123.21332740783691
1 번째 데이터 한개당 걸린 시간 : 0.06160666370391846
예측한 값과 가까운 아이템 5개의 Accuracy : 0.954
----------------------------------------------------
2 번째 추천 시작
2 번째 추천(2000건)에 걸린 시간(초): 120.60031747817993
2 번째 데이터 한개당 걸린 시간 : 0.06030015873908996
예측한 값과 가까운 아이템 5개의 Accuracy : 0.9435
----------------------------------------------------
3 번째 추천 시작
3 번째 추천(2000건)에 걸린 시간(초): 120.84765791893005
3 번째 데이터 한개당 걸린 시간 : 0.06042382895946503
예측한 값과 가까운 아이템 5개의 Accuracy : 0.941
----------------------------------------------------
4 번째 추천 시작
4 번째 추천(2000건)에 걸린 시간(초): 118.93876552581787
4 번째 데이터 한개당 걸린 시간 : 0.05946938276290894
예측한 값과 가까운 아이템 5개의 Accuracy : 0.951
----------------------------------------------------
5 번째 추천 시작
5 번째 추천(2000건)에 걸린 시간(초): 122.44438576698303
5 번째 데이터 한개당 걸린 시간 : 0.06122219288349152
예측한 값과 가까운 아이템 5개의 Accuracy : 0.936
----------------------------------------------------
6 번째 추천 시작
6 번째 추천(2000건)에 걸린 시간(초): 119.56845474243164
6 번째 데이터 한개당 